# Value Investing Indicators from SEC Filings

### Data Source: https://www.sec.gov/dera/data/financial-statement-data-sets.html



In [1]:
import pandas as pd
import os
import shutil
import glob
import sys
import warnings
warnings.simplefilter("ignore")

os.chdir("..")

dir_root = os.getcwd()


In [2]:
dir_raw = dir_root + u"/sec_filings/Raw"

cik_lookup = pd.read_csv(dir_root + u"/sec_filings/cik_ticker.csv", usecols=['CIK', 'Ticker'], sep="|")


In [3]:
num_tags = ["PreferredStockValue", "AssetsCurrent", "Liabilities", "EarningsPerShareBasic", "CommonStockSharesOutstanding", "LiabilitiesCurrent", "EarningsPerShareBasic", "SharePrice", "StockholdersEquity", "PreferredStockValue", "CommonStockSharesOutstanding", "NetIncomeLoss", "GrossProfit", "SalesRevenueNet","StockRepurchasedAndRetiredDuringPeriodShares"]

num_cols = ['adsh', 'tag', 'uom', 'value']

sub_cols = ['adsh', 'ein', 'cik', 'sic', 'name', 'period', 'fp']

files_num = sorted(glob.glob(dir_raw + u'/*_num.txt'))

files_sub = sorted(glob.glob(dir_raw + u'/*_sub.txt'))

sec_df = pd.DataFrame([])

for num, sub in zip(files_num, files_sub):
    df_num = pd.read_csv(num, sep='\t', dtype=str, encoding = "ISO-8859-1")
    df_sub = pd.read_csv(sub, sep='\t', dtype=str, encoding = "ISO-8859-1")
    df_sub = df_sub[sub_cols]
    df_num = df_num[num_cols]
    df_num = df_num[df_num['tag'].isin(num_tags)]
    sec_merge = df_sub.merge(df_num, on='adsh', sort=True)
    sec_df = sec_df.append(sec_merge)


In [20]:
cik_lookup.columns = cik_lookup.columns.str.lower()

sec_df['ticker'] = sec_df['cik'].map(cik_lookup.set_index('cik')['ticker'].to_dict())

sec_df['cik'] = sec_df['cik'].astype(str)

cik_lookup['cik'] = cik_lookup['cik'].astype(str)

sec_df = sec_df.dropna(subset=['ticker'])

In [41]:
sec_df = sec_df.drop_duplicates(subset=['adsh', 'tag'])

sec_df.set_index(['ticker']).unstack().reset_index()

sec_df

,level_0,ticker,0
0,adsh,FRX,0000038074-09-000029
1,adsh,FRX,0000038074-09-000029
2,adsh,FRX,0000038074-09-000029
3,adsh,FRX,0000038074-09-000029
4,adsh,FRX,0000038074-09-000029
...,...,...,...
12465995,value,NIHK,0.0000
12465996,value,NIHK,139153206.0000
12465997,value,NIHK,0.0000
12465998,value,NIHK,0.0000


# Benjamin Graham 

## Formulas:

* NCAVPS = CurrentAssets - (Total Liabilities + Preferred Stock) ÷ Shares Outstanding
    * Less than 1.10 

* Debt to Assets = Current Assets / Current Liabilities
    * Greater than 1.50

* Price / Earnings per Share ratio 
    * Less than 9.0

* PRICE TO BOOK VALUE = (P/BV) 
    * Where BV = (Total Shareholder Equity−Preferred Stock)/ Total Outstanding Shares
    * Less than 1.20. P/E ratios

## References:

Benjamin Graham rules: https://cabotwealth.com/daily/value-investing/benjamin-grahams-value-stock-criteria/ 

Benjamin Graham rules Modified: https://www.netnethunter.com/16-benjamin-graham-rules/ 



In [4]:

sec_df['NCAVPS'] = sec_df['AssetsCurrent'] - (sec_df['Liabilities'] + sec_df[
'PreferredStockValue']) / sec_df['CommonStockSharesOutstanding']

sec_df['DebtToAssets'] = sec_df['AssetsCurrent'] / sec_df['LiabilitiesCurrent']

sec_df['PE'] = sec_df['SharePrice'] / sec_df['EarningsPerShareBasic'] 

sec_df['PBV'] = sec_df['SharePrice'] / ((sec_df['StockholdersEquity'] - sec_df['PreferredStockValue']) / sec_df['CommonStockSharesOutstanding'])


SyntaxError: invalid syntax (<ipython-input-4-168e8a96d945>, line 8)

# Warren Buffet Rules 

## Formulas

* Debt/Equity= Total Liabilities / Total Shareholders’ Equity 
    * Less than 1 and ROE is greater than 10%

* Return on Earnings = (Net Income / Stock Holders Equity)
    * Is Positive

* Gross Profit Margin = Gross Profit / Revenue 
    * Greater than 40% 
​
* Quarter over Quarter EPS 
    * Greater than 10

* Stock Buybacks
    * Greater than last period

## References: 
https://www.oldschoolvalue.com/tutorial/this-is-how-buffett-interprets-financial-statements/

In [ ]:

sec_df['DebtEquity'] = sec_df['Liabilities'] / sec_df['StockholdersEquity']

sec_df['ReturnEarnings'] = sec_df['NetIncomeLoss'] / sec_df['StockholdersEquity']

sec_df["GrossProfitMargin"] = sec_df['GrossProfit'] / sec_df['SalesRevenueNet']

sec_df["EPS"] = sec_df["EarningsPerShareBasic"]

sec_df["StockBuybacks"] = sec_df["StockRepurchasedAndRetiredDuringPeriodShares"]